In [1]:
# get data

# get pos and ner

# make pred with lime

# reduce pos and ner to lime features

In [2]:
%cd ..
from src.model_wrapper import Model
from src.explainer_lime import Lime_Explanation

C:\Users\Stefan.Beuchert\Desktop\backup_from_kubectl


In [3]:
import pandas as pd
#pd.options.mode.chained_assignment = None
#import numpy as np
#import os
#import logging

In [40]:
# config
chunk_tag = 'A' # the possible chunks are A, B, C, D, E with 2700 rows each
target_path_for_combined_data = f'./data/GermanQuAD_raw_combined/GermanQuAD_raw_combined_{chunk_tag}.json'
model_name = 'deepset/gelectra-base-germanquad'

number_of_features = 10 # the default value
number_of_samples = 5000 # the default value

In [41]:
# get raw data combined in one dataframe (test + train)
GQuAD_chunk_data = pd.read_json(target_path_for_combined_data)
GQuAD_chunk_data = GQuAD_chunk_data.head(1)
GQuAD_chunk_data

,question_id,question,answers,context,document_id,is_impossible,usage
0,67118,Warum sind höhere Temperaturen für die Bildung...,"[{'answer_id': 72940, 'document_id': 73863, 'q...",Gletscher\n\n=== Akkumulation und Metamorphose...,73863,False,train


In [42]:
# get important instances
GELECTRA_MODEL = Model(model_name)

In [43]:
q = GQuAD_chunk_data.at[0, 'question']
c = GQuAD_chunk_data.at[0, 'context']

output, all_tokens, encoding = GELECTRA_MODEL.predict(q, c)

In [44]:
answer = GELECTRA_MODEL.get_predicted_tokens(output, all_tokens)
answer

[('Zum', 0.808849573135376),
 ('.', 0.462344229221344),
 ("['Zum', 'einen', 'bilden', 'sich', 'in', 'wär', '##mere', '##n', '(', 'tem', '##per', '##ierten', ')', 'Gletscher', '##n', 'in', 'der', 'Regel', 'kleinere', 'Eis', '##kr', '##ista', '##lle', ',', 'wodurch', 'hier', 'und', 'auch', 'in', 'den', 'Vor', '##stufen', 'des', 'Eis', '##es', 'wie', 'Fir', '##n', 'und', 'gra', '##nu', '##lar', '##em', 'Eis', '(', 'in', 'mancher', 'Literatur', 'auch', 'Fir', '##nei', '##s', 'genannt', ')', 'eine', 'Bewegung', 'möglich', 'ist', ',', 'bei', 'der', 'leichter', 'Luft', 'frei', '##gesetzt', 'werden', 'kann', '.', 'Darüber', 'hinaus', 'kann', 'auch', 'ober', '##fläch', '##liches', 'Material', 'aufs', '##ch', '##melzen', 'und', 'erneut', 'gefr', '##ieren', ',', 'ohne', 'den', 'Gletscher', 'zu', 'verlassen', '.', 'So', 'kann', 'zumindest', 'in', 'kleineren', 'Mengen', 'sogar', 'im', 'Tages', '##zyklus', 'eine', 'Verwand', '##lung', 'von', 'Schnee', 'in', 'Eis', 'stattfinden', ',', 'ohne', 'dass',

In [45]:
answer_string = GELECTRA_MODEL.get_answer_string(output, all_tokens)
answer_string

'Zum einen bilden sich in wärmeren ( temperierten ) Gletschern in der Regel kleinere Eiskristalle, wodurch hier und auch in den Vorstufen des Eises wie Firn und granularem Eis ( in mancher Literatur auch Firneis genannt ) eine Bewegung möglich ist, bei der leichter Luft freigesetzt werden kann. Darüber hinaus kann auch oberflächliches Material aufschmelzen und erneut gefrieren, ohne den Gletscher zu verlassen. So kann zumindest in kleineren Mengen sogar im Tageszyklus eine Verwandlung von Schnee in Eis stattfinden, ohne dass die bei der Druckmetamorphose üblichen Zwischenstufen durchlaufen werden.'

In [46]:
output.start_logits

tensor([[-0.4111, -5.7883, -6.0279, -5.9497, -6.5005, -6.1438, -6.2551, -6.1799,
         -6.4551, -6.5137, -6.2048, -6.6743, -6.4466, -0.4111, -6.2217, -6.2402,
         -6.4177, -6.5605, -5.9955, -6.2413, -6.5037, -6.3010, -5.9604, -6.2322,
         -6.3260, -6.5193, -6.0466, -6.3931, -6.2979, -4.8877, -6.0402, -6.2898,
         -5.8971, -6.3335, -6.3958, -6.3488, -6.7794, -6.1110, -5.5385, -6.2029,
         -6.5835, -6.6768, -6.3447, -5.8310, -6.3918, -5.7715, -6.0136, -6.3034,
         -6.3847, -6.3348, -6.3613, -6.4674, -6.7661, -6.3734, -5.7974, -6.2122,
         -6.3809, -6.4090, -6.4917, -6.7563, -6.3032, -5.9464, -6.4755, -6.4755,
         -6.6538, -5.5082, -0.1149, -4.6661, -5.3820, -5.9086, -6.1481, -6.2814,
         -5.6483, -5.0435, -5.1040, -5.9499, -5.5832, -6.2759, -6.0619, -6.2011,
         -5.8007, -5.8081, -6.3415, -6.4413, -6.0964, -6.2828, -6.4647, -6.4872,
         -6.2516, -5.4938, -5.8271, -5.6480, -6.0487, -6.0882, -6.3945, -6.3310,
         -6.3711, -6.7242, -

In [47]:
print(output[0][0].shape[0]) # start_token logits
print(output[1][0].shape[0]) # end_token logits
print(len(all_tokens))
print(len(encoding.input_ids))
print(len(encoding.token_type_ids))
print(len(encoding.attention_mask))
print(all_tokens)

368
368
368
368
368
368
['[CLS]', 'Warum', 'sind', 'höhere', 'Temperaturen', 'für', 'die', 'Bildung', 'der', 'Gletscher', 'vorteil', '##haft', '?', '[SEP]', 'Gletscher', '=', '=', '=', 'Ak', '##kum', '##ulation', 'und', 'Met', '##amo', '##rp', '##hose', '=', '=', '=', 'Der', 'Prozess', 'der', 'Ans', '##ammlung', 'von', 'Schnee', '##massen', 'wird', 'Ak', '##kum', '##ulation', 'genannt', 'und', 'infolge', '##dessen', 'der', 'Entstehung', '##sb', '##erei', '##ch', 'eines', 'Gletscher', '##s', 'mit', 'Ak', '##kum', '##ulation', '##sg', '##eb', '##iet', '(', 'Nähr', '##gebiet', ')', 'bezeichnet', '.', 'Reicht', 'die', 'Schnee', '##mächtig', '##keit', 'aus', ',', 'dass', 'durch', 'die', 'Auf', '##last', 'der', 'oberen', 'die', 'tiefer', '##en', 'Schichten', 'zusammenge', '##press', '##t', 'werden', ',', 'beginnt', 'die', 'Met', '##amo', '##rp', '##hose', 'des', 'Schnee', '##s', 'hin', 'zu', 'Gletscher', '##eis', '.', 'Dabei', 'wird', 'durch', 'den', 'in', 'der', 'Tiefe', 'immer', 'höher', '